<a href="https://colab.research.google.com/github/nurshatfateh/YouTube-Textual-Summarization-using-Google-T5-and-SpaCy/blob/main/YouTube_Textual_Summarization_using_Google_T5_%26_SpaCy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install evaluate
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 24.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/sim

In [ ]:
import evaluate
rouge = evaluate.load('rouge')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**data processing**

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/train.csv", usecols=['highlights', 'article'], nrows=5000)

In [ ]:
df

,article,highlights
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...
4995,"(CNN) -- Despite being recently branded a ""rot...",Mario Balotelli returns to Italy joining AC Mi...
4996,London (CNN) -- Let's play spot the villain. O...,Several priests have recently told kids that S...
4997,A jealous husband who tried to strangle his nu...,Royston Jones was convinced that wife Claire J...
4998,Defence bosses were last night embroiled in a ...,Author 'forced to resign' from the Army after ...


In [ ]:
df = df.rename(columns={"highlights":"target_text", "article":"source_text"})
df = df[['source_text', 'target_text']]

In [ ]:
df.head()

,source_text,target_text
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
df['source_text'] = "summarize: " + df['source_text']
df.head()

,source_text,target_text
0,summarize: By . Associated Press . PUBLISHED: ...,"Bishop John Folda, of North Dakota, is taking ..."
1,summarize: (CNN) -- Ralph Mata was an internal...,Criminal complaint: Cop used his role to help ...
2,summarize: A drunk driver who killed a young w...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,summarize: (CNN) -- With a breezy sweep of his...,Nina dos Santos says Europe must be ready to a...
4,summarize: Fleetwood are the only team still t...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((4000, 2), (1000, 2))

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

**Google T5 model tuning**

In [ ]:
! pip install simplet5 -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 79.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 59.5.0 which is incompatible.
cvxpy 1.3.1 requires setu

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
size=train_df['source_text'].size
size

4000

In [ ]:
model.train(train_df=train_df[:4000],
            eval_df=test_df[:100],
            source_max_token_len=512,
            target_max_token_len=150,
            batch_size=16, max_epochs=5, use_gpu=True)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.seed:Global seed set to 42
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
! ( cd outputs; ls )

simplet5-epoch-0-train-loss-1.6647-val-loss-1.3717
simplet5-epoch-1-train-loss-1.4928-val-loss-1.3778
simplet5-epoch-2-train-loss-1.3795-val-loss-1.3883
simplet5-epoch-3-train-loss-1.2833-val-loss-1.4176
simplet5-epoch-4-train-loss-1.195-val-loss-1.4465


In [ ]:
model.load_model("t5","/content/outputs/simplet5-epoch-4-train-loss-1.195-val-loss-1.4465", use_gpu=True)

**Youtube video to text processing**

In [ ]:
!pip install youtube-transcript-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
link = "https://www.youtube.com/watch?v=reUZRyXxUs4"
id = link.split("=")[-1]
id

'reUZRyXxUs4'

In [ ]:
pretext=YouTubeTranscriptApi.get_transcript(id)
pretext

[{'text': 'When I think about the rise of AI,',
  'start': 4.376,
  'duration': 3.086},
 {'text': "I'm reminded by the rise of literacy.",
  'start': 7.504,
  'duration': 3.17},
 {'text': 'A few hundred years ago,', 'start': 10.715, 'duration': 1.21},
 {'text': 'many people in society thought',
  'start': 11.967,
  'duration': 1.543},
 {'text': 'that maybe not everyone needed\nto be able to read and write.',
  'start': 13.552,
  'duration': 3.92},
 {'text': 'Back then, many people were\ntending fields or herding sheep,',
  'start': 17.514,
  'duration': 2.919},
 {'text': 'so maybe there was less need\nfor written communication.',
  'start': 20.433,
  'duration': 2.67},
 {'text': 'And all that was needed', 'start': 23.144, 'duration': 1.168},
 {'text': 'was for the high priests\nand priestesses and monks',
  'start': 24.354,
  'duration': 2.377},
 {'text': 'to be able to read the Holy Book,',
  'start': 26.773,
  'duration': 1.585},
 {'text': 'and the rest of us could just go\nto the te

In [ ]:
text=""
for i in pretext:
  s=i["text"]
  text=text+s+" "
text

'When I think about the rise of AI, I\'m reminded by the rise of literacy. A few hundred years ago, many people in society thought that maybe not everyone needed\nto be able to read and write. Back then, many people were\ntending fields or herding sheep, so maybe there was less need\nfor written communication. And all that was needed was for the high priests\nand priestesses and monks to be able to read the Holy Book, and the rest of us could just go\nto the temple or church or the holy building and sit and listen to the high priest\nand priestesses read to us. Fortunately, it was since figured out\nthat we can build a much richer society if lots of people can read and write. Today, AI is in the hands\nof the high priests and priestesses. These are the highly skilled AI engineers, many of whom work\nin the big tech companies. And most people have access\nonly to the AI that they build for them. I think that we can build\na much richer society if we can enable everyone\nto help to write

In [ ]:
finaltext="summarize:"+text
finaltext


'summarize:When I think about the rise of AI, I\'m reminded by the rise of literacy. A few hundred years ago, many people in society thought that maybe not everyone needed\nto be able to read and write. Back then, many people were\ntending fields or herding sheep, so maybe there was less need\nfor written communication. And all that was needed was for the high priests\nand priestesses and monks to be able to read the Holy Book, and the rest of us could just go\nto the temple or church or the holy building and sit and listen to the high priest\nand priestesses read to us. Fortunately, it was since figured out\nthat we can build a much richer society if lots of people can read and write. Today, AI is in the hands\nof the high priests and priestesses. These are the highly skilled AI engineers, many of whom work\nin the big tech companies. And most people have access\nonly to the AI that they build for them. I think that we can build\na much richer society if we can enable everyone\nto hel

**Google T5 Abstractive Summarization Result**

In [ ]:
t5_result=model.predict(finaltext)
t5_result

Token indices sequence length is longer than the specified maximum sequence length for this model (2331 > 512). Running this sequence through the model will result in indexing errors


["Today, AI is in the hands of the high priests and priestesses. We can build a much richer society if we enable everyone to help to write the future. In the coming era for AI, we'll empower everyone to build AI systems for themselves."]

**--------------OPTIONAL SECTION STARTS--------------**

**SpaCy Preprocessing**

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
doc = nlp(text)

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)

In [ ]:
from string import punctuation
punctuation = punctuation + '\n'

In [ ]:
wordfreq = {}
for word in doc:
  if word.text.lower() not in stopwords:
    if word.text.lower() not in punctuation:
      if word.text not in wordfreq.keys():
        wordfreq[word.text] = 1
      else:
        wordfreq[word.text] += 1

In [ ]:
for word in wordfreq.keys():
  wordfreq[word] = wordfreq[word]/max(wordfreq.values())
wordfreq

{'think': 0.1,
 'rise': 0.04,
 'AI': 1.0,
 'reminded': 0.07692307692307693,
 'literacy': 0.23076923076923078,
 'years': 0.23076923076923078,
 'ago': 0.15384615384615385,
 'people': 0.7692307692307693,
 'society': 0.3076923076923077,
 'thought': 0.07692307692307693,
 'maybe': 0.3076923076923077,
 'needed': 0.15384615384615385,
 'able': 0.38461538461538464,
 'read': 0.3076923076923077,
 'write': 0.46153846153846156,
 'tending': 0.07692307692307693,
 'fields': 0.07692307692307693,
 'herding': 0.07692307692307693,
 'sheep': 0.07692307692307693,
 'need': 0.23076923076923078,
 'written': 0.07692307692307693,
 'communication': 0.07692307692307693,
 'high': 0.3076923076923077,
 'priests': 0.23076923076923078,
 'priestesses': 0.3076923076923077,
 'monks': 0.07692307692307693,
 'Holy': 0.07692307692307693,
 'Book': 0.07692307692307693,
 'rest': 0.07692307692307693,
 'temple': 0.07692307692307693,
 'church': 0.07692307692307693,
 'holy': 0.07692307692307693,
 'building': 0.15384615384615385,
 'si

In [ ]:
senttokens = [i for i in doc.sents]
senttokens

[When I think about the rise of AI, I'm reminded by the rise of literacy.,
 A few hundred years ago, many people in society thought that maybe not everyone needed
 to be able to read and write.,
 Back then, many people were
 tending fields or herding sheep, so maybe there was less need
 for written communication.,
 And all that was needed was for the high priests
 and priestesses and monks to be able to read the Holy Book, and the rest of us could just go
 to the temple or church or the holy building and sit and listen to the high priest
 and priestesses read to us.,
 Fortunately, it was since figured out
 that we can build a much richer society if lots of people can read and write.,
 Today, AI is in the hands
 of the high priests and priestesses.,
 These are the highly skilled AI engineers, many of whom work
 in the big tech companies.,
 And most people have access
 only to the AI that they build for them.,
 I think that we can build
 a much richer society if we can enable everyone
 t

In [ ]:
sentweight = {}
for sent in senttokens:
  for word in sent:
    if word.text.lower() in wordfreq.keys():
      if sent not in sentweight.keys():
        sentweight[sent] = wordfreq[word.text.lower()]
      else:
        sentweight[sent] += wordfreq[word.text.lower()]
sentweight

{When I think about the rise of AI, I'm reminded by the rise of literacy.: 0.4876923076923077,
 A few hundred years ago, many people in society thought that maybe not everyone needed
 to be able to read and write.: 3.153846153846154,
 Back then, many people were
 tending fields or herding sheep, so maybe there was less need
 for written communication.: 1.7692307692307692,
 And all that was needed was for the high priests
 and priestesses and monks to be able to read the Holy Book, and the rest of us could just go
 to the temple or church or the holy building and sit and listen to the high priest
 and priestesses read to us.: 3.4615384615384617,
 Fortunately, it was since figured out
 that we can build a much richer society if lots of people can read and write.: 3.0769230769230775,
 Today, AI is in the hands
 of the high priests and priestesses.: 1.5230769230769232,
 These are the highly skilled AI engineers, many of whom work
 in the big tech companies.: 2.3076923076923075,
 And most p

**SpaCy Extractive Summarization Result**

In [ ]:
from heapq import nlargest
sizee=int(len(senttokens)*0.1)
summ=nlargest(sizee, sentweight, key=sentweight.get)
summ

[Just as pen and paper, which are a vastly superior technology
 to stone tablet and chisel, were instrumental to widespread literacy, there are emerging new
 AI development platforms that shift the focus from asking you
 to write lots of code to asking you to focus on providing data.,
 And I say to you, this is a big deal to that inspector
 whose life this makes easier and equally, this type of technology
 can empower a baker to use AI to check for the quality
 of the cakes they're making, or an organic farmer to check
 the quality of the vegetables, or a furniture maker to check
 the quality of the wood they're using.,
 So an inspector using
 an accessible platform like this can, in a few hours to a few days, and with purchasing
 a suitable camera set up, be able to build a custom AI system
 to detect defects, tears and discolorations in all the fabric being used to make T-shirts
 throughout the factory.,
 I hope that the pizzeria owner and many other small
 business owners like him w

In [ ]:
spacy_res = [word.text for word in summ]
print(spacy_res)

['Just as pen and paper, which are a vastly superior technology\nto stone tablet and chisel, were instrumental to widespread literacy, there are emerging new\nAI development platforms that shift the focus from asking you\nto write lots of code to asking you to focus on providing data.', "And I say to you, this is a big deal to that inspector\nwhose life this makes easier and equally, this type of technology\ncan empower a baker to use AI to check for the quality\nof the cakes they're making, or an organic farmer to check\nthe quality of the vegetables, or a furniture maker to check\nthe quality of the wood they're using.", 'So an inspector using\nan accessible platform like this can, in a few hours to a few days, and with purchasing\na suitable camera set up, be able to build a custom AI system\nto detect defects, tears and discolorations in all the fabric being used to make T-shirts\nthroughout the factory.', 'I hope that the pizzeria owner and many other small\nbusiness owners like h

In [ ]:
spacy_result = ' '.join(spacy_res)
print(spacy_result)

Just as pen and paper, which are a vastly superior technology
to stone tablet and chisel, were instrumental to widespread literacy, there are emerging new
AI development platforms that shift the focus from asking you
to write lots of code to asking you to focus on providing data. And I say to you, this is a big deal to that inspector
whose life this makes easier and equally, this type of technology
can empower a baker to use AI to check for the quality
of the cakes they're making, or an organic farmer to check
the quality of the vegetables, or a furniture maker to check
the quality of the wood they're using. So an inspector using
an accessible platform like this can, in a few hours to a few days, and with purchasing
a suitable camera set up, be able to build a custom AI system
to detect defects, tears and discolorations in all the fabric being used to make T-shirts
throughout the factory. I hope that the pizzeria owner and many other small
business owners like him will also take advant

**--------------OPTIONAL SECTION ENDS--------------**

**Google T5 Evaluation**

In [ ]:
test_df.iloc[:,0].values[0]

"summarize: By . William Cook, Andy Dolan, Nick Fagge, Neil Sears and Paul Bentley . PUBLISHED: . 09:46 EST, 30 October 2012 . | . UPDATED: . 20:14 EST, 30 October 2012 . Police are investigating allegations that a BBC radio journalist killed himself after being sexually harassed by a female colleague. Russell Joslin, 50, whose father is a former chief constable, suffocated himself last Monday despite being on suicide watch at a psychiatric hospital. His family claim he was driven to his death after bosses at the BBC failed to take seriously allegations that he was being bullied by a female colleague. Gathering information: Police are assisting in establishing the circumstances surrounding the death of Russell Joslin, pictured left in 2008 and right, before his death in a psychiatric hospital last Monday . The woman is said to have targeted . him after he spurned her sexual advances during a night out at a . restaurant in 2007 near his home in affluent Kenilworth, Warwickshire. She all

In [ ]:
t55=model.predict(test_df.iloc[:,0].values[0])
t55[0]

'Russell Joslin, 50, was admitted to a psychiatric hospital in Warwickshire on suicide watch last Monday. His family claim he was sexually harassed and bullied by a female colleague at the station. The former chief constable has called for an investigation into his treatment.'

In [ ]:
reff=test_df.iloc[:,1].values[0]
type(reff)
reff

"Russell Joslin, 50, found dead after being admitted to psychiatric hospital following previous suicide attempt .\nFamily of BBC radio reporter say he was bullied by colleague after spurning her sexual advances .\nHis father Peter, a former chief constable, asked police to investigate allegations .\nWarwickshire Police confirm they are assisting in 'establishing the circumstances' of death ."

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

In [ ]:
scores = scorer.score(reff,t55[0])

In [ ]:
scores

{'rouge1': Score(precision=0.5454545454545454, recall=0.43636363636363634, fmeasure=0.48484848484848486)}